In [1]:
from graph import Dataset, Graph

In [2]:
dataset = Dataset()

data_dir = 'data/FB15k-237'
dataset.set_id2node(f'{data_dir}/ind2ent.pkl')
dataset.set_id2rel(f'{data_dir}/ind2rel.pkl')
dataset.set_node2title(f'{data_dir}/extra/entity2text.txt')

Loaded 14505 nodes from data/FB15k-237/ind2ent.pkl.
Loaded 474 relations from data/FB15k-237/ind2rel.pkl.
Loaded 14951 node titles from data/FB15k-237/extra/entity2text.txt.


In [3]:
graph_train = Graph(dataset)
graph_train.load_triples(f'{data_dir}/train.txt', skip_missing=False, add_reverse=True)
graph_train.get_num_nodes(), graph_train.get_num_edges()

(14505, 544230)

In [4]:
graph_valid = Graph(dataset)
# add training edges to validation graph
for edge in graph_train.get_edges():
    # we set add_reverse=False because it already exists in the training graph
    graph_valid.add_edge(edge.get_head().get_name(), edge.get_name(), edge.get_tail().get_name(), skip_missing=False, add_reverse=False)
graph_valid.load_triples(f'{data_dir}/valid.txt', skip_missing=False, add_reverse=True)
graph_valid.get_num_nodes(), graph_valid.get_num_edges()

(14505, 579300)

In [5]:
graph_test = Graph(dataset)
# add training and validation edges to test graph (validation graph contains all training edges)
for edge in graph_valid.get_edges():
    # we set add_reverse=False because it already exists in the validation graph
    graph_test.add_edge(edge.get_head().get_name(), edge.get_name(), edge.get_tail().get_name(), skip_missing=False, add_reverse=False)
graph_test.load_triples(f'{data_dir}/test.txt', skip_missing=False, add_reverse=True)
graph_test.get_num_nodes(), graph_test.get_num_edges()

(14505, 620232)

In [6]:
from query import Query, QueryDataset, human_readable

In [7]:
from symbolic import SymbolicReasoning

In [8]:
dir_query_2p = 'data/FB15k-237/test_ans_2c.pkl'
dir_query_3p = 'data/FB15k-237/test_ans_3c.pkl'
dir_query_2i = 'data/FB15k-237/test_ans_2i.pkl'
dir_query_2u = 'data/FB15k-237/test_ans_2u.pkl'
dir_query_3i = 'data/FB15k-237/test_ans_3i.pkl'
dir_query_pi = 'data/FB15k-237/test_ans_ci.pkl'
dir_query_ip = 'data/FB15k-237/test_ans_ic.pkl'
dir_query_up = 'data/FB15k-237/test_ans_uc.pkl'

query_dataset = QueryDataset(dataset)
query_dataset.load_queries_from_pkl(dir_query_2p, query_type='2p')
query_dataset.load_queries_from_pkl(dir_query_3p, query_type='3p')
query_dataset.load_queries_from_pkl(dir_query_2i, query_type='2i')
query_dataset.load_queries_from_pkl(dir_query_2u, query_type='2u')
query_dataset.load_queries_from_pkl(dir_query_3i, query_type='3i')
query_dataset.load_queries_from_pkl(dir_query_pi, query_type='pi')
query_dataset.load_queries_from_pkl(dir_query_ip, query_type='ip')
query_dataset.load_queries_from_pkl(dir_query_up, query_type='up')
query_dataset.get_num_queries()

40000

In [9]:
sample_idx = 4000
sample_query_type = '2p'
query = query_dataset.get_queries(sample_query_type)[sample_idx]
human_readable(query, dataset)

Query:
Lamar Odom	--/education/educational_institution/students_graduates./education/education/student_reverse-->	V
V	--/education/educational_degree/people_with_this_degree./education/education/institution_reverse-->	?

Answer Set (?): 
['Bachelor of Science', 'PhD', 'Doctorate', 'Bachelor of Arts', "Bachelor's degree"]


In [10]:
query_dataset.get_num_queries()

40000

In [11]:
dir_query_2p = 'data/FB15k-237/test_ans_2c_hard.pkl'
dir_query_3p = 'data/FB15k-237/test_ans_3c_hard.pkl'
dir_query_2i = 'data/FB15k-237/test_ans_2i_hard.pkl'
dir_query_2u = 'data/FB15k-237/test_ans_2u_hard.pkl'
dir_query_3i = 'data/FB15k-237/test_ans_3i_hard.pkl'
dir_query_pi = 'data/FB15k-237/test_ans_ci_hard.pkl'
dir_query_ip = 'data/FB15k-237/test_ans_ic_hard.pkl'
dir_query_up = 'data/FB15k-237/test_ans_uc_hard.pkl'

query_dataset_hard = QueryDataset(dataset)
query_dataset_hard.load_queries_from_pkl(dir_query_2p, query_type='2p')
query_dataset_hard.load_queries_from_pkl(dir_query_3p, query_type='3p')
query_dataset_hard.load_queries_from_pkl(dir_query_2i, query_type='2i')
query_dataset_hard.load_queries_from_pkl(dir_query_2u, query_type='2u')
query_dataset_hard.load_queries_from_pkl(dir_query_3i, query_type='3i')
query_dataset_hard.load_queries_from_pkl(dir_query_pi, query_type='pi')
query_dataset_hard.load_queries_from_pkl(dir_query_ip, query_type='ip')
query_dataset_hard.load_queries_from_pkl(dir_query_up, query_type='up')
query_dataset_hard.get_num_queries()

40000

In [12]:
sample_query_type = '2p'
sample_idx = 4000
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
human_readable(query_hard, dataset)

Query:
Lamar Odom	--/education/educational_institution/students_graduates./education/education/student_reverse-->	V
V	--/education/educational_degree/people_with_this_degree./education/education/institution_reverse-->	?

Answer Set (?): 
['Doctorate']


In [13]:
def accuracy(query: Query, answers: list):
    correct_answers = set(query.get_answer())
    predicted_answers = set(answers)
    if len(correct_answers) == 0:
        return 0.0
    return len(correct_answers.intersection(predicted_answers)) / len(correct_answers)

In [14]:
reasoner_train = SymbolicReasoning(graph_train)

sample_idx = 4000
query = query_dataset.get_queries(sample_query_type)[sample_idx]
human_readable(query, dataset)

middle_steps, answers_train = reasoner_train.query_2p(query.get_query()[0][0], query.get_query()[0][1])
print(f"Answers from train graph: {middle_steps}")
print(f"Final Answers: {answers_train}")
print(f"Expected Answers: {query.get_answer()}")
print(f"Accuracy: {accuracy(query, answers_train)}")

Query:
Lamar Odom	--/education/educational_institution/students_graduates./education/education/student_reverse-->	V
V	--/education/educational_degree/people_with_this_degree./education/education/institution_reverse-->	?

Answer Set (?): 
['Bachelor of Science', 'PhD', 'Doctorate', 'Bachelor of Arts', "Bachelor's degree"]
Querying for head: Lamar Odom (12324 | /m/02_nkp) and relation: /education/educational_institution/students_graduates./education/education/student_reverse (45)
Found edge: Lamar Odom --/education/educational_institution/students_graduates./education/education/student_reverse--> University of Rhode Island (4074)
Found edge: Lamar Odom --/education/educational_institution/students_graduates./education/education/student_reverse--> University of Nevada, Las Vegas (9463)
--------------------------------------------------
Querying for head: University of Rhode Island (4074 | /m/02fjzt) and relation: /education/educational_degree/people_with_this_degree./education/education/i

In [15]:
reasoner_test = SymbolicReasoning(graph_test)

middle_steps, answers_test = reasoner_test.query_2p(query.get_query()[0][0], query.get_query()[0][1])
print(f"Answers from test graph: {middle_steps}")
print(f"Final Answers: {answers_test}")
print(f"Expected Answers: {query.get_answer()}")
print(f"Accuracy: {accuracy(query, answers_test)}")

Querying for head: Lamar Odom (12324 | /m/02_nkp) and relation: /education/educational_institution/students_graduates./education/education/student_reverse (45)
Found edge: Lamar Odom --/education/educational_institution/students_graduates./education/education/student_reverse--> University of Rhode Island (4074)
Found edge: Lamar Odom --/education/educational_institution/students_graduates./education/education/student_reverse--> University of Nevada, Las Vegas (9463)
--------------------------------------------------
Querying for head: University of Rhode Island (4074 | /m/02fjzt) and relation: /education/educational_degree/people_with_this_degree./education/education/institution_reverse (179)
Found edge: University of Rhode Island --/education/educational_degree/people_with_this_degree./education/education/institution_reverse--> PhD (587)
Found edge: University of Rhode Island --/education/educational_degree/people_with_this_degree./education/education/institution_reverse--> Bachelor o

In [16]:
reasoner_test.fixed_size_answer(answers_test, 10)

,score
706,1.0
587,1.0
3181,1.0
1177,1.0
1566,1.0
8762,0.0
6871,0.0
6163,0.0
5858,0.0
11264,0.0


In [17]:
reasoner_train = SymbolicReasoning(graph_train, logging=False)
reasoner_valid = SymbolicReasoning(graph_valid, logging=False)
reasoner_test = SymbolicReasoning(graph_test, logging=False)

answers_train = reasoner_train.query_2p(query.get_query()[0][0], query.get_query()[0][1])[1]
answers_valid = reasoner_valid.query_2p(query.get_query()[0][0], query.get_query()[0][1])[1]
answers_test = reasoner_test.query_2p(query.get_query()[0][0], query.get_query()[0][1])[1]

print(f"Train Accuracy: {accuracy(query, answers_train)}")
print(f"Valid Accuracy: {accuracy(query, answers_valid)}")
print(f"Test Accuracy: {accuracy(query, answers_test)}")

Train Accuracy: 0.8
Valid Accuracy: 0.8
Test Accuracy: 1.0


In [18]:
from cqd import create_cqd_file

In [19]:
create_cqd_file([query], output_file='data/FB15k-237/FB15k-237_test_hard_sample.pkl')

Creating CQD file: 100%|██████████| 1/1 [00:00<00:00, 14716.86it/s]


In [20]:
from cqd import cqd_query, get_cache_prediction

In [21]:
cqd_query(query, sample_path='data/FB15k-237/FB15k-237_test_hard_sample.pkl', result_path='scores.json', k=5)

Creating CQD file: 100%|██████████| 1/1 [00:00<00:00, 24105.20it/s]


ComplEx(
  (embeddings): ModuleList(
    (0): Embedding(14505, 2000, sparse=True)
    (1): Embedding(474, 2000, sparse=True)
  )
)


100%|██████████| 1/1 [00:00<00:00, 16.61it/s]


Saving results to scores.json


,score
9463,10.657951
4074,10.455688
7265,6.088031
4683,5.748784
1236,5.670690


In [22]:
import pandas as pd

cqd_cache = pd.read_json('data/FB15k-237/all_1p_queries_top25.json', orient='records')
cqd_cache

,entity_id,relation_id,top_k_entities,top_k_scores
0,8227,402,"[1964, 7094, 5233, 3593, 6775, 4762, 7548, 577...","[1.641523718833923, 1.5853451490402222, 1.5796..."
1,8227,403,"[5145, 12787, 400, 7208, 4947, 9367, 12456, 11...","[6.992974281311035, 4.854063987731934, 4.71984..."
2,8227,404,"[1527, 7216, 4168, 226, 4994, 592, 32, 4331, 1...","[9.617685317993164, 4.58512020111084, 4.580133..."
3,8227,405,"[2037, 7859, 1779, 7421, 8288, 1104, 6393, 226...","[1.398288369178772, 1.334656715393066, 1.30624..."
4,8227,406,"[10948, 6607, 9650, 4005, 4, 863, 3202, 6386, ...","[1.6829309463500972, 1.536678314208984, 1.4762..."
...,...,...,...,...
6875365,1265,385,"[8410, 1265, 2328, 1266, 1568, 5790, 2327, 104...","[8.260475158691406, 7.059150695800781, 6.96028..."
6875366,1265,386,"[62, 4, 11, 163, 23, 344, 212, 706, 529, 9, 39...","[2.077280521392822, 1.39265489578247, 1.258444..."
6875367,1265,387,"[470, 862, 657, 6676, 818, 215, 845, 330, 8400...","[1.121290802955627, 1.064321041107177, 1.04515..."
6875368,1265,388,"[1265, 2328, 8410, 1266, 1568, 3842, 5790, 565...","[4.588125228881836, 3.088305473327636, 2.88703..."


In [23]:
cqd_query(query, k=25, cqd_cache=cqd_cache)

,score
9463,10.657951
4074,10.455688
7265,6.088031
4683,5.748784
1236,5.670690
3169,5.650109
2173,5.561179
6483,5.536769
5153,5.439585
3895,5.423999


## Example Usage

In [24]:
sample_idx = 4000
query = query_dataset.get_queries(sample_query_type)[sample_idx]
human_readable(query, dataset)

Query:
Lamar Odom	--/education/educational_institution/students_graduates./education/education/student_reverse-->	V
V	--/education/educational_degree/people_with_this_degree./education/education/institution_reverse-->	?

Answer Set (?): 
['Bachelor of Science', 'PhD', 'Doctorate', 'Bachelor of Arts', "Bachelor's degree"]


In [25]:
symbolic = SymbolicReasoning(graph_train, logging=False)

In [26]:
sample_idx = 4000
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
query = query_dataset.get_queries(sample_query_type)[sample_idx]
easy_answers = query.get_answer()
easy_answers = [a for a in easy_answers if a not in query_hard.get_answer()]
human_readable(query_hard, dataset)
print("-"*70)
human_readable(query, dataset)

Query:
Lamar Odom	--/education/educational_institution/students_graduates./education/education/student_reverse-->	V
V	--/education/educational_degree/people_with_this_degree./education/education/institution_reverse-->	?

Answer Set (?): 
['Doctorate']
----------------------------------------------------------------------
Query:
Lamar Odom	--/education/educational_institution/students_graduates./education/education/student_reverse-->	V
V	--/education/educational_degree/people_with_this_degree./education/education/institution_reverse-->	?

Answer Set (?): 
['Bachelor of Science', 'PhD', 'Doctorate', 'Bachelor of Arts', "Bachelor's degree"]


In [27]:
query_hard

Query(type=2p, query=((12324, (45, 179)),), answer=[3181])

In [28]:
query

Query(type=2p, query=((12324, (45, 179)),), answer=[706, 587, 3181, 1177, 1566])

In [29]:
from xcqa import XCQA

symbolic = SymbolicReasoning(graph_train, logging=False)
xcqa = XCQA(symbolic = symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [30]:
xcqa.query_execution(query,k=10, coalition=[0, 0], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.07 seconds
Time taken for second level query: 0.71 seconds


,score,path
entity_id,,
587,1.0,12324--45-->4074--179-->
706,1.0,12324--45-->4074--179-->
1177,1.0,12324--45-->4074--179-->
1566,1.0,12324--45-->9463--179-->
0,0.0,NaN
...,...,...
14500,0.0,NaN
14501,0.0,NaN
14502,0.0,NaN


In [31]:
xcqa.query_execution(query, k=10, coalition=[0, 1], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.20 seconds


,score,path
entity_id,,
1177,8.299611,12324--45-->4074--179-->
706,8.166074,12324--45-->4074--179-->
1566,7.979342,12324--45-->9463--179-->
587,7.124403,12324--45-->4074--179-->
1019,6.903434,12324--45-->4074--179-->
...,...,...
14500,0.000000,NaN
14501,0.000000,NaN
14502,0.000000,NaN


In [32]:
xcqa.query_execution(query, k=10, coalition=[0, 1], t_norm='min', t_conorm='min')

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds


,score,path
entity_id,,
484,1.0,12324--45-->4074--179-->
587,1.0,12324--45-->4074--179-->
706,1.0,12324--45-->4074--179-->
1019,1.0,12324--45-->4074--179-->
1177,1.0,12324--45-->4074--179-->
...,...,...
14500,0.0,NaN
14501,0.0,NaN
14502,0.0,NaN


In [33]:
xcqa. query_execution(query, k=10, coalition=[1, 0], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.55 seconds


,score,path
entity_id,,
1177,10.657951,12324--45-->9463--179-->
1566,10.657951,12324--45-->9463--179-->
587,10.455688,12324--45-->4074--179-->
706,10.455688,12324--45-->4074--179-->
1019,6.088031,12324--45-->7265--179-->
...,...,...
14500,0.000000,NaN
14501,0.000000,NaN
14502,0.000000,NaN


In [34]:
xcqa.query_execution(query,  k=10, coalition=[1, 1], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.16 seconds


,score,path
entity_id,,
1177,86.778140,12324--45-->4074--179-->
706,85.381916,12324--45-->4074--179-->
1566,85.043434,12324--45-->9463--179-->
587,74.490537,12324--45-->4074--179-->
1019,72.180147,12324--45-->4074--179-->
...,...,...
14500,0.000000,NaN
14501,0.000000,NaN
14502,0.000000,NaN


In [35]:
from shapley import value_function

In [36]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [37]:
value_function(xcqa, query, easy_answers, target_entity=0, qoi='rank', k=10, coalition=[0, 0], t_norm='prod', t_conorm='min')

0

In [38]:
value_function(xcqa, query, easy_answers, target_entity=0, qoi='rank', k=10, coalition=[0, 1], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.07 seconds
Time taken for second level query: 0.19 seconds


7

In [39]:
value_function(xcqa, query, easy_answers, target_entity=0, qoi='rank', k=10, coalition=[1, 0], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.69 seconds


12

In [40]:
value_function(xcqa, query, easy_answers, target_entity=0, qoi='rank', k=10, coalition=[1, 1], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.16 seconds


12

In [41]:
from shapley import shapley_value

In [42]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

Shapley value of the first player (atom 0):

In [43]:
shapley_value(xcqa, query, atom_idx=0, easy_answers=easy_answers, target_entity=query_hard.answer[0], qoi='rank', k=10, t_norm='prod', t_conorm='min')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.68 seconds
Coalition: [0, 0], Contribution: 4 (before adding atom: 0, after adding atom: 4), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.07 seconds
Time taken for second level query: 0.19 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.16 seconds
Coalition: [0, 1], Contribution: 0 (before adding atom: 4, after adding atom: 4), weight: 0.5)
Shapley value for atom 0: 2.0


2.0

Shapley value of the second player (atom 1):

In [44]:
shapley_value(xcqa, query, atom_idx=1, easy_answers=easy_answers, target_entity=query_hard.answer[0], qoi='rank', k=10, t_norm='prod', t_conorm='min')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 4 (before adding atom: 0, after adding atom: 4), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds


Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: 0 (before adding atom: 4, after adding atom: 4), weight: 0.5)
Shapley value for atom 1: 2.0


2.0

Value function when every atom is present in the coalition:

In [45]:
value_function(xcqa, query, easy_answers, target_entity=query_hard.answer[0], qoi='rank', k=10, coalition=[1, 1], t_norm='prod', t_conorm='min')

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds


4

## Shapley Value Examples

### Shapley Value for 2p

In [46]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [47]:
sample_idx = 1000
sample_query_type = '2p'
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
query = query_dataset.get_queries(sample_query_type)[sample_idx]
easy_answers = query.get_answer()
easy_answers = [a for a in easy_answers if a not in query_hard.get_answer()]
human_readable(query_hard, dataset)
print("-"*70)
human_readable(query, dataset)

Query:
Perkin Medal	--/award/award_category/winners./award/award_honor/award_winner-->	V
V	--/people/ethnicity/people_reverse-->	?

Answer Set (?): 
['African American']
----------------------------------------------------------------------
Query:
Perkin Medal	--/award/award_category/winners./award/award_honor/award_winner-->	V
V	--/people/ethnicity/people_reverse-->	?

Answer Set (?): 
['African American']


In [48]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1], t_norm='prod', t_conorm='prod')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.02 seconds


Time taken for second level query: 0.19 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
1253,59.608866,14483--94-->4165--143-->,Irish American,False,False
879,59.601220,14483--94-->4165--143-->,Jewish people,False,False
11038,55.189793,14483--94-->4165--143-->,Mexican American,False,False
3324,55.045102,14483--94-->4165--143-->,White American,False,False
2279,53.064225,14483--94-->4165--143-->,African American,False,True
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


Let's explain the true hard answer which the model could predict it correctly.

In [49]:
target = 'African American'
target_entity = dataset.get_node_by_title(target)
target_id = dataset.get_id_by_node(target_entity)
print(f"Target entity: {target} ({target_entity}) with ID {target_id}")

Target entity: African American (/m/0x67) with ID 2279


In [50]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.69 seconds
Coalition: [0, 0], Contribution: 2282 (before adding atom: 0, after adding atom: 2282), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.07 seconds
Time taken for second level query: 0.17 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: 0 (before adding atom: 4, after adding atom: 4), weight: 0.5)
Shapley value for atom 0: 1141.0


1141.0

In [51]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 4 (before adding atom: 0, after adding atom: 4), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -2278 (before adding atom: 2282, after adding atom: 4), weight: 0.5)
Shapley value for atom 1: -1137.0


-1137.0

Now, let's look at the explanation for another target entity, which is not in the answer set of the query.

In [52]:
target = "Italian American"
target_entity = dataset.get_node_by_title(target)
target_id = dataset.get_id_by_node(target_entity)
print(f"Target entity: {target} ({target_entity}) with ID {target_id}")

Target entity: Italian American (/m/0xnvg) with ID 3330


In [53]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 3333 (before adding atom: 0, after adding atom: 3333), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: 0 (before adding atom: 7, after adding atom: 7), weight: 0.5)
Shapley value for atom 0: 1666.5


1666.5

In [54]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 7 (before adding atom: 0, after adding atom: 7), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -3326 (before adding atom: 3333, after adding atom: 7), weight: 0.5)
Shapley value for atom 1: -1659.5


-1659.5

Let's look at the explanation when we use `min` as t-norm.

In [55]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1], t_norm='min', t_conorm='max')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
1253,5.624588,14483--94-->4165--143-->,Irish American,False,False
879,5.623867,14483--94-->4165--143-->,Jewish people,False,False
11038,5.207612,14483--94-->4165--143-->,Mexican American,False,False
3324,5.193959,14483--94-->4165--143-->,White American,False,False
2279,5.007047,14483--94-->4165--143-->,African American,False,True
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


In [56]:
target = 'African American'
target_entity = dataset.get_node_by_title(target)
target_id = dataset.get_id_by_node(target_entity)
print(f"Target entity: {target} ({target_entity}) with ID {target_id}")

Target entity: African American (/m/0x67) with ID 2279


In [57]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 2282 (before adding atom: 0, after adding atom: 2282), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: 0 (before adding atom: 4, after adding atom: 4), weight: 0.5)
Shapley value for atom 0: 1141.0


1141.0

In [58]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 4 (before adding atom: 0, after adding atom: 4), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -2278 (before adding atom: 2282, after adding atom: 4), weight: 0.5)
Shapley value for atom 1: -1137.0


-1137.0

### Shapley Value for 3p

In [59]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [60]:
sample_query_type = '3p'
sample_idx = 1000
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
query = query_dataset.get_queries(sample_query_type)[sample_idx]
easy_answers = query.get_answer()
easy_answers = [a for a in easy_answers if a not in query_hard.get_answer()]
human_readable(query_hard, dataset)
print("-"*70)
human_readable(query, dataset)

Query:
Winston-Salem	--/location/hud_county_place/place_reverse-->	V1
V1	--/base/biblioness/bibs_location/country-->	V2
V2	--/location/statistical_region/places_exported_to./location/imports_and_exports/exported_to-->	?

Answer Set (?): 
['Angola', 'Anguilla', 'Australia', 'Netherlands', 'Afghanistan', 'Japan', 'Singapore', 'Sierra Leone', 'Germany', 'Algeria', 'Brazil', 'China', 'Mexico', 'Taiwan', 'Belgium', 'South Korea', 'Switzerland', 'Dubai', 'United Kingdom']
----------------------------------------------------------------------
Query:
Winston-Salem	--/location/hud_county_place/place_reverse-->	V1
V1	--/base/biblioness/bibs_location/country-->	V2
V2	--/location/statistical_region/places_exported_to./location/imports_and_exports/exported_to-->	?

Answer Set (?): 
['Angola', 'Anguilla', 'Australia', 'Netherlands', 'Afghanistan', 'Japan', 'Singapore', 'Sierra Leone', 'Germany', 'Algeria', 'Brazil', 'China', 'Mexico', 'Taiwan', 'Belgium', 'South Korea', 'Switzerland', 'Dubai', 'Unit

In [61]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[0, 0, 0], t_norm='prod', t_conorm='min')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.07 seconds
Time taken for second level query: 7.94 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
0,0.0,NaN,Dominican Republic,False,False
1,0.0,NaN,Republic,False,False
2,0.0,5561--113-->13529--266-->12314--436-->,Mighty Morphin Power Rangers,False,False
3,0.0,NaN,Wendee Lee,False,False
4,0.0,NaN,Drama,False,False
...,...,...,...,...,...
14500,0.0,5561--113-->308--266-->2122--436-->,"Strategic Simulations, Inc.",False,False
14501,0.0,5561--113-->5561--266-->9611--436-->,House of Plantagenet,False,False
14502,0.0,NaN,Humour,False,False


In [62]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1, 1], t_norm='prod', t_conorm='min')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.73 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
5138,620.009017,5561--113-->5561--266-->32--436-->,Anguilla,False,True
6150,613.922489,5561--113-->5561--266-->32--436-->,Angola,False,True
3100,608.222374,5561--113-->5561--266-->32--436-->,Afghanistan,False,True
190,601.928367,5561--113-->5561--266-->32--436-->,Algeria,False,True
32,594.299743,5561--113-->5561--266-->32--436-->,United States of America,False,False
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


In [63]:
target = 'Anguilla'
target_entity = dataset.get_node_by_title(target)
target_id = dataset.get_id_by_node(target_entity)
print(f"Target entity: {target} ({target_entity}) with ID {target_id}")

Target entity: Anguilla (/m/0n3g) with ID 5138


In [64]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='min')

Coalition: [0, 0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 6.66 seconds
Coalition: [0, 0, 0], Contribution: 5140 (before adding atom: 0, after adding atom: 5140), weight: 0.3333333333333333)
Coalition: [0, 0, 1], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 1.95 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 1.64 seconds
Coalition: [0, 0, 1], Contribution: -5138 (before adding atom: 5138, after adding atom: 0), weight: 0.16666666666666666)
Coalition: [0, 1, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 3.55 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.67 seconds
Coalition: [0, 1, 0], Contribution: 1 (before adding atom: 5144, after adding atom: 5145), weight: 0.16666666666666666)
Coalition: [0, 1, 1], Atom Index: 0
Time taken for first 

857.1666666666666

In [65]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='min')

Coalition: [0, 0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.05 seconds
Coalition: [0, 0, 0], Contribution: 5144 (before adding atom: 0, after adding atom: 5144), weight: 0.3333333333333333)
Coalition: [0, 0, 1], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Coalition: [0, 0, 1], Contribution: -5138 (before adding atom: 5138, after adding atom: 0), weight: 0.16666666666666666)
Coalition: [1, 0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Coalition: [1, 0, 0], Contribution: 5 (before adding atom: 5140, after adding atom: 5145), weight: 0.16666666666666666)
Coalition: [1, 0, 1], Atom Index: 1
Time taken for first 

859.1666666666666

In [66]:
atom_idx = 2
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='min')

Coalition: [0, 0, 0], Atom Index: 2
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Coalition: [0, 0, 0], Contribution: 5138 (before adding atom: 0, after adding atom: 5138), weight: 0.3333333333333333)
Coalition: [0, 1, 0], Atom Index: 2
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Coalition: [0, 1, 0], Contribution: -5144 (before adding atom: 5144, after adding atom: 0), weight: 0.16666666666666666)
Coalition: [1, 0, 0], Atom Index: 2
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.04 seconds
Coalition: [1, 0, 0], Contribution: -5140 (before adding atom: 5140, after adding atom: 0), weight: 0.16666666666666666)
Coalition: [1, 1, 0], Atom Index: 2
Time taken for first

-1716.3333333333335

### Shapley Value for 2u

In [67]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [68]:
sample_query_type = '2u'
sample_idx = 40
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
query = query_dataset.get_queries(sample_query_type)[sample_idx]
easy_answers = query.get_answer()
easy_answers = [a for a in easy_answers if a not in query_hard.get_answer()]
human_readable(query_hard, dataset)
print("-"*70)
human_readable(query, dataset)

Query:
Michel Legrand	--/people/person/profession-->	V1
Vitamin B-12	--/food/food/nutrients./food/nutrition_fact/nutrient_reverse-->	V2
V1	OR	V2	-->	?

Answer Set (?): 
['Conductor', 'Chicken meat']
----------------------------------------------------------------------
Query:
Michel Legrand	--/people/person/profession-->	V1
Vitamin B-12	--/food/food/nutrients./food/nutrition_fact/nutrient_reverse-->	V2
V1	OR	V2	-->	?

Answer Set (?): 
['Composer', 'Pasta', 'Ice cream', 'Beef', 'Conductor', 'Chicken meat', 'Milk', 'Pork', 'Egg', 'Cheese', 'Pianist-GB']


In [69]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1], t_norm='prod', t_conorm='prod')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.02 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
10329,7.837002,0\n13927--275-->,Pork,True,False
5883,7.825286,0\n13927--275-->,Egg,True,False
5756,7.823646,0\n13927--275-->,Cheese,True,False
10844,7.803871,3873--34-->\n0,Pianist-GB,True,False
9450,7.675278,0\n13927--275-->,Pasta,True,False
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


In [70]:
target = 'Chicken meat'
target_entity = dataset.get_node_by_title(target)
target_id = dataset.get_id_by_node(target_entity)
print(f"Target entity: {target} ({target_entity}) with ID {target_id}")

Target entity: Chicken meat (/m/0f25w9) with ID 1746


In [71]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [72]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.07 seconds
Coalition: [0, 0], Contribution: 1749 (before adding atom: 0, after adding atom: 1749), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.07 seconds
Time taken for second level query: 0.02 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: 0 (before adding atom: 0, after adding atom: 0), weight: 0.5)
Shapley value for atom 0: 874.5


874.5

In [73]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 0 (before adding atom: 0, after adding atom: 0), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -1749 (before adding atom: 1749, after adding atom: 0), weight: 0.5)
Shapley value for atom 1: -874.5


-874.5

In [74]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1], t_norm='min', t_conorm='max')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
10329,7.837002,0\n13927--275-->,Pork,True,False
5883,7.825286,0\n13927--275-->,Egg,True,False
5756,7.823646,0\n13927--275-->,Cheese,True,False
10844,7.803871,3873--34-->\n0,Pianist-GB,True,False
9450,7.675278,0\n13927--275-->,Pasta,True,False
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


In [75]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='min', t_conorm='max')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 1749 (before adding atom: 0, after adding atom: 1749), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: 0 (before adding atom: 0, after adding atom: 0), weight: 0.5)
Shapley value for atom 0: 874.5


874.5

In [76]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='min', t_conorm='max')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 0 (before adding atom: 0, after adding atom: 0), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -1749 (before adding atom: 1749, after adding atom: 0), weight: 0.5)
Shapley value for atom 1: -874.5


-874.5

### Shapley Value for 2i

In [77]:
xcqa = XCQA(symbolic=symbolic, dataset=dataset, cqd_cache=cqd_cache, logging=True)

In [78]:
sample_query_type = '2i'
sample_idx = 1000
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
query = query_dataset.get_queries(sample_query_type)[sample_idx]
easy_answers = query.get_answer()
easy_answers = [a for a in easy_answers if a not in query_hard.get_answer()]
human_readable(query_hard, dataset)
print("-"*70)
human_readable(query, dataset)

Query:
Roots reggae	--/music/genre/parent_genre-->	V1
Red Hot Chili Peppers	--/music/genre/artists_reverse-->	V2
V1	AND	V2	-->	?

Answer Set (?): 
['Reggae']
----------------------------------------------------------------------
Query:
Roots reggae	--/music/genre/parent_genre-->	V1
Red Hot Chili Peppers	--/music/genre/artists_reverse-->	V2
V1	AND	V2	-->	?

Answer Set (?): 
['Reggae', 'Ska']


In [79]:
query

Query(type=2i, query=((13677, (104,)), (7779, (41,))), answer=[184, 5825])

In [80]:
def answer_readable(answers, dataset):
    return [dataset.get_title_by_node(dataset.get_node_by_id(a)) for a in answers]

In [81]:
answer_readable(symbolic.query_1p(13677, 104), dataset)

['Ska']

In [82]:
answer_readable(symbolic.query_1p(7779, 41), dataset)

['Ska',
 'Classic rock',
 'Funk metal',
 'Psychedelic rock',
 'Alternative rock',
 'Rap rock',
 'Reggae',
 'Punk rock',
 'Funk rock',
 'Hard rock']

In [83]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[0, 0], t_norm='prod', t_conorm='prod')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.07 seconds
Time taken for second level query: 0.06 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
5825,1.0,13677--104-->\n7779--41-->,Ska,True,False
0,0.0,NaN,Dominican Republic,False,False
1,0.0,NaN,Republic,False,False
2,0.0,NaN,Mighty Morphin Power Rangers,False,False
3,0.0,NaN,Wendee Lee,False,False
...,...,...,...,...,...
14500,0.0,NaN,"Strategic Simulations, Inc.",False,False
14501,0.0,NaN,House of Plantagenet,False,False
14502,0.0,NaN,Humour,False,False


In [84]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1], t_norm='prod', t_conorm='prod')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.02 seconds
Time taken for second level query: 0.02 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
5825,75.102507,13677--104-->\n7779--41-->,Ska,True,False
5658,38.702916,13677--104-->\n7779--41-->,Punk rock,False,False
0,0.000000,NaN,Dominican Republic,False,False
1,0.000000,NaN,Republic,False,False
2,0.000000,NaN,Mighty Morphin Power Rangers,False,False
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


In [85]:
target = 'Punk rock'
target_entity = dataset.get_node_by_title(target)
target_id = dataset.get_id_by_node(target_entity)
print(f"Target entity: {target} ({target_entity}) with ID {target_id}")

Target entity: Punk rock (/m/05r6t) with ID 5658


In [86]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 1 (before adding atom: 0, after adding atom: 1), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: -5658 (before adding atom: 5658, after adding atom: 0), weight: 0.5)
Shapley value for atom 0: -2828.5


-2828.5

In [87]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='prod', t_conorm='prod')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 5658 (before adding atom: 0, after adding atom: 5658), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -1 (before adding atom: 1, after adding atom: 0), weight: 0.5)
Shapley value for atom 1: 2828.5


2828.5

In [88]:
cqd_result = xcqa.query_execution(query, k=10, coalition=[1, 1], t_norm='min', t_conorm='max')
human_df = cqd_result.copy()
human_df['title'] = human_df.index.map(dataset.id2node)
human_df['title'] = human_df['title'].map(dataset.get_title_by_node)
human_df['is_easy_answer'] = human_df.index.isin(easy_answers)
human_df['is_hard_answer'] = human_df.index.isin(query_hard.get_answer())
human_df

Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds


,score,path,title,is_easy_answer,is_hard_answer
entity_id,,,,,
5825,7.169825,13677--104-->\n7779--41-->,Ska,True,False
5658,5.042561,13677--104-->\n7779--41-->,Punk rock,False,False
0,0.000000,NaN,Dominican Republic,False,False
1,0.000000,NaN,Republic,False,False
2,0.000000,NaN,Mighty Morphin Power Rangers,False,False
...,...,...,...,...,...
14500,0.000000,NaN,"Strategic Simulations, Inc.",False,False
14501,0.000000,NaN,House of Plantagenet,False,False
14502,0.000000,NaN,Humour,False,False


In [89]:
atom_idx = 0
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='min', t_conorm='max')

Coalition: [0, 0], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 1 (before adding atom: 0, after adding atom: 1), weight: 0.5)
Coalition: [0, 1], Atom Index: 0
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 1], Contribution: -5658 (before adding atom: 5658, after adding atom: 0), weight: 0.5)
Shapley value for atom 0: -2828.5


-2828.5

In [90]:
atom_idx = 1
shapley_value(xcqa, query, atom_idx=atom_idx, easy_answers=easy_answers, target_entity=target_id, qoi='rank', k=10, t_norm='min', t_conorm='max')

Coalition: [0, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [0, 0], Contribution: 5658 (before adding atom: 0, after adding atom: 5658), weight: 0.5)
Coalition: [1, 0], Atom Index: 1
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Time taken for first level query: 0.00 seconds
Time taken for second level query: 0.00 seconds
Coalition: [1, 0], Contribution: -1 (before adding atom: 1, after adding atom: 0), weight: 0.5)
Shapley value for atom 1: 2828.5


2828.5

### Shapley Value for 3i

In [91]:
sample_query_type = '3i'
sample_idx = 1000
query_hard = query_dataset_hard.get_queries(sample_query_type)[sample_idx]
query = query_dataset.get_queries(sample_query_type)[sample_idx]
easy_answers = query.get_answer()
easy_answers = [a for a in easy_answers if a not in query_hard.get_answer()]
human_readable(query_hard, dataset)
print("-"*70)
human_readable(query, dataset)

Query:
Tom Noonan	--/people/person/places_lived./people/place_lived/location-->	V1
Brendan Fraser	--/people/person/places_lived./people/place_lived/location-->	V2
Tom Noonan	--/people/person/places_lived./people/place_lived/location-->	V3
V1	AND	V2	AND	V3	-->	?

Answer Set (?): 
['Greenwich']
----------------------------------------------------------------------
Query:
Tom Noonan	--/people/person/places_lived./people/place_lived/location-->	V1
Brendan Fraser	--/people/person/places_lived./people/place_lived/location-->	V2
Tom Noonan	--/people/person/places_lived./people/place_lived/location-->	V3
V1	AND	V2	AND	V3	-->	?

Answer Set (?): 
['Greenwich']
